# Package

In [ ]:
import xpress as xp
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import itertools
from itertools import chain
import warnings
warnings.filterwarnings("ignore")

# Pre-defined function

In [ ]:
def pattern_comb(Length_of_items,Length_of_the_object):
    Length_of_items = list(int(x) for x in Length_of_items)
    c = Length_of_items.copy()
    for i in Length_of_items:
        mult = 2
        while i*mult <= Length_of_the_object:
            c.append(i)
            mult += 1
    rlist = []
    for i in range(1,len(c)+1):
        exec(f'rlist{i} = [i for i in np.unique([sorted(list(i)) for i in itertools.combinations(c, i)],axis=0) if sum(i) <= {Length_of_the_object}]')
        rlist.append(eval(f'rlist{i}'))
    return list(chain.from_iterable(rlist))

def pattern_heuristic(Length_of_items,Length_of_the_object):
    
    # Building the base patterns
    rlist = []
    [rlist.append(np.array([i])) for i in Length_of_items]
    for i in Length_of_items:
        mult = 2
        while i*mult <= Length_of_the_object:
            rlist.append(np.array(np.repeat(i,mult)))
            mult += 1
    
    # 2 to 1 combinations
    for i in range(len(Length_of_items)):
        for j in range(i+1,len(Length_of_items)):
            if Length_of_items[i]+Length_of_items[j] <= Length_of_the_object:
                rlist.append(np.array([Length_of_items[i],Length_of_items[j]]))
    
    # 3 to 1 combinations
    for i in range(len(Length_of_items)):
        for j in range(len(Length_of_items)):
            for k in range(j+1,len(Length_of_items)):
                if Length_of_items[i]+Length_of_items[j]+Length_of_items[k] <= Length_of_the_object:
                    rlist.append(np.array([Length_of_items[i],Length_of_items[j],Length_of_items[k]]))
    
    # Make sure no duplicates
    rlist = [np.array(i) for i in list(np.unique([list(i) for i in [np.sort(i) for i in rlist]]))]
    
    return rlist

# Reading data

In [ ]:
group_num = 1
example_num = 1

In [ ]:
# Reading all required components from the data
with open(f'data/group{group_num}/example{example_num}.txt', mode="r", encoding="utf-8") as fp:
    info_data = fp.read()

number_of_decision_periods = int(re.findall('periods.*?(\d+)\n', info_data)[0])

number_of_item_types = int(re.findall('types.*?(\d+)\n',info_data)[0])

number_of_affected_periods = int(re.findall('affected\speriods.*?(\d+)\n',info_data)[0])

affect_periods = re.findall('Affected periods:\t(\d+.*\d+)\t\n',info_data)[0].split('\t')
affected_periods = np.array([int(i) for i in affect_periods])

Length_of_the_object  = int(re.findall('Length.*?object:\t(\d+).*',info_data)[0])

leng_items =re.findall('Length.*items:\t(\d+.*\d+).*',info_data)[0].split('\t')
Length_of_items = np.array([int(j) for j in leng_items])

Predict_demand = re.findall('Predicted.*items:\n(\d+.*\d+).*Realistic',info_data,re.S)[0].split('\t')
Predicted_demand_of_items = np.array([int(k) for k in Predict_demand]).reshape(number_of_decision_periods,number_of_item_types)

Real_demand = re.findall('Realistic.*items:\n(\d+.*\d+).*Consumed',info_data,re.S)[0].split('\t')
Realistic_demand_of_items = np.array([int(l) for l in Real_demand]).reshape(number_of_decision_periods,number_of_item_types)

Consumed_cost_per_object  = int(re.findall('Consumed.*?object:\t(\d+).*', info_data)[0])

Pattern_setup_cost = int(re.findall('Pattern\ssetup.*?:\t(\d+).*', info_data)[0])

Pattern_deviation_cost = int(re.findall('Pattern\sdeviation.*?:\t(\d+).*', info_data)[0])

Period_dev = re.findall('Period\sdeviation.*:\t(\d.*\d).*',info_data)[0].split('\t')
Period_deviation_cost = np.array([float(m) for m in Period_dev])

Inventory_hold = re.findall('Inventory.*item:\t(\d.*\d).*',info_data)[0].split('\t')
Inventory_holding_cost_per_item = np.array([float(o) for o in Inventory_hold])

num_ava_objects = re.findall('.*available.*period:\t(\d.*\d).*',info_data)[0].split('\t')
Number_of_available_objects_in_each_period  = np.array([float(u) for u in num_ava_objects])

In [ ]:
# Find all possible patterns (Full optimization)
pattern_combination = pattern_comb(Length_of_items,Length_of_the_object)

In [ ]:
# Pattern combinations using heuristics (Pattern reduced version)
pattern_combination = pattern_heuristic(Length_of_items,Length_of_the_object)

In [ ]:
# Construct the a matrix
number_of_possible_pattern = len(pattern_combination)
a =np.array([0*Length_of_items])
for i in pattern_combination:
    number_of_each_pattern = np.array([0*Length_of_items])
    for j in range(number_of_item_types):
        for k in i:
            if Length_of_items[j] == k:
                number_of_each_pattern[0,j] += 1
    a=np.concatenate((a,number_of_each_pattern),axis=0)
a = a[1:]


# Define required variables
items = range(number_of_item_types)
decision_periods = range(number_of_decision_periods)
pattern = range(number_of_possible_pattern)
h = Inventory_holding_cost_per_item
epsilon = Period_deviation_cost
l = Length_of_items

# CSPS

In [ ]:
def csps(period):
    # Note that period must be specified as integer starting from 0
    
    # csps problem
    prob = xp.problem(name='csps')
    prob.controls.outputlog = 0
    
    # Decision variables
    x = np.array([xp.var(name="x_{0}".format(j+1),lb = 0, ub = xp.infinity,
                    vartype=xp.integer)for j in pattern],dtype=xp.npvar)
    y = np.array([xp.var(name="y_{0}".format(j+1),vartype=xp.binary)
                  for j in pattern],dtype=xp.npvar)
    prob.addVariable(x,y)
    
    # Convert notation as shown in the article
    C = Consumed_cost_per_object 
    beta = Pattern_setup_cost
    d = Predicted_demand_of_items[period,:]
    L = Length_of_the_object
    l = Length_of_items.copy()
    Q = Number_of_available_objects_in_each_period[period]
    
    # Obj function
    obj = xp.Sum(C*x[j]+beta*y[j] for j in pattern)
    prob.setObjective(obj,sense = xp.minimize)
    
    # Define constraints and add them to the model
    prob.addConstraint(xp.Sum(a[j,i]*x[j] for j in pattern) == d[i] for i in items)
    prob.addConstraint(xp.Sum(l[i]*a[j,i] for i in items) <= L for j in pattern)
    prob.addConstraint(x[j] <= Q*y[j] for j in pattern)
    
    prob.addConstraint(y[j] <= x[j] for j in pattern)
    
    prob.addConstraint(xp.Sum(x[j] for j in pattern) <= Q)
    
    # solve problem
    prob.solve()
    
    # Store the solution in decision space
    x_out = prob.getSolution(x)
    y_out = prob.getSolution(y)
    
    # Store the solution in objective space
    f_sol = xp.evaluate(obj,problem=prob)
    
    return x_out,y_out,f_sol

# CSRPS

In [ ]:
def csrps(period):
    # Note that period must be specified as integer starting from 0
    
    # csrps problem
    prob = xp.problem(name='csrps')
    prob.controls.outputlog = 0
    
    # Decision variables
    x = np.array([xp.var(name="x_{0}".format(j+1),lb = 0, ub = xp.infinity,
                    vartype=xp.integer)for j in pattern],dtype=xp.npvar)
    y = np.array([xp.var(name="y_{0}".format(j+1),vartype=xp.binary)
                  for j in pattern],dtype=xp.npvar)
    phi = np.array([xp.var(name="phi_{0}".format(j+1),vartype=xp.binary)
                  for j in pattern],dtype=xp.npvar)
    prob.addVariable(x,y,phi)
    
    # Convert notation as shown in the article
    C = Consumed_cost_per_object
    beta = Pattern_setup_cost
    d = Realistic_demand_of_items[period,:]
    L = Length_of_the_object
    l = Length_of_items.copy()
    Q = Number_of_available_objects_in_each_period[period]
    eta = Pattern_deviation_cost
    
    # Optimal y* obtained from csps
    y_star = csps(period)[1]
    
    # Obj function
    obj = xp.Sum(C*x[j]+beta*y[j]+eta*phi[j] for j in pattern)
    prob.setObjective(obj,sense = xp.minimize)
    
    # Define constraints and add them to the model
    prob.addConstraint(xp.Sum(a[j,i]*x[j] for j in pattern) == d[i] for i in items)
    prob.addConstraint(xp.Sum(l[i]*a[j,i] for i in items) <= L for j in pattern)
    prob.addConstraint(x[j] <= Q*y[j] for j in pattern)
    
    prob.addConstraint(y[j] <= x[j] for j in pattern)
    
    prob.addConstraint(xp.Sum(x[j] for j in pattern) <= Q)
    prob.addConstraint(phi[j] >= y[j] - y_star[j] for j in pattern)
    prob.addConstraint(phi[j] >= y_star[j] - y[j] for j in pattern)
    
    # solve problem
    prob.solve()
    
    # Store the solution in decision space
    x_out = prob.getSolution(x)
    y_out = prob.getSolution(y)
    phi_out = prob.getSolution(phi)
    
    # Store the solution in objective space
    f_sol = xp.evaluate(obj,problem=prob)
    
    return x_out,y_out,phi_out,f_sol

# MCSRPS

In [ ]:
def mcsrps():
    
    # mcsrps problem
    prob = xp.problem(name='mcsrps')
    prob.controls.outputlog = 0
    
    # Decision variables
    x = np.array([xp.var(name="x_{0}_{1}".format(j+1,t+1),lb = 0, ub = xp.infinity,
                    vartype=xp.integer)for j in pattern for t in decision_periods],
                 dtype=xp.npvar).reshape(number_of_possible_pattern,number_of_decision_periods)
    y = np.array([xp.var(name="y_{0}_{1}".format(j+1,t+1),vartype=xp.binary)
                  for j in pattern for t in decision_periods],
                 dtype=xp.npvar).reshape(number_of_possible_pattern,number_of_decision_periods)
    phi = np.array([xp.var(name="phi_{0}_{1}".format(j+1,t+1),vartype=xp.binary)
                  for j in pattern for t in decision_periods],
                   dtype=xp.npvar).reshape(number_of_possible_pattern,number_of_decision_periods)
    I = np.array([xp.var(name="I_{0}_{1}".format(i+1,t+1),lb = 0, ub = xp.infinity,
                    vartype=xp.integer)for i in items for t in decision_periods],
                 dtype=xp.npvar).reshape(number_of_item_types,number_of_decision_periods)
    r = np.array([xp.var(name="r_{0}".format(t+1),vartype=xp.binary)
                for t in decision_periods],
                 dtype=xp.npvar)
    prob.addVariable(x,y,phi,I,r)
    
    # Convert notation as shown in the article
    C = Consumed_cost_per_object 
    beta = Pattern_setup_cost
    d = Realistic_demand_of_items.T
    L = Length_of_the_object
    l = Length_of_items.copy()
    Q = Number_of_available_objects_in_each_period
    eta = Pattern_deviation_cost
    
    # Optimal y* obtained from csps
    y_star = np.array([])
    for i in range(number_of_decision_periods):
        y_star = np.append(y_star,csps(i)[1].round())
    y_star = y_star.reshape(number_of_decision_periods,number_of_possible_pattern).T
    
    # Obj function
    obj = xp.Sum(C * x[j,t] + beta * y[j,t] + eta * phi[j,t] for j in pattern for t in decision_periods) + \
          xp.Sum(h[i] * I[i,t] for i in items for t in decision_periods) + \
          xp.Sum(epsilon[t] * r[t] for t in decision_periods)
    prob.setObjective(obj,sense = xp.minimize)
    
    # Define constraints and add them to the model
    prob.addConstraint(I[i,0] == xp.Sum(a[j,i]*x[j,0] for j in pattern) - d[i,0] for i in items)
    prob.addConstraint(I[i,t] == xp.Sum(a[j,i]*x[j,t] for j in pattern) + I[i,t-1] - d[i,t] for i in items for t in decision_periods if t != 0)
    prob.addConstraint(xp.Sum(l[i]*a[j,i] for i in items) <= L for j in pattern)
    prob.addConstraint(x[j,t] <= Q[t] * y[j,t] for j in pattern for t in decision_periods)
    
    prob.addConstraint(y[j,t] <= x[j,t] for j in pattern for t in decision_periods)
    
    prob.addConstraint(xp.Sum(x[j,t] for j in pattern) <= Q[t] for t in decision_periods)
    prob.addConstraint(phi[j,t] >= y[j,t] - y_star[j,t] for j in pattern for t in decision_periods)
    prob.addConstraint(phi[j,t] >= y_star[j,t] - y[j,t] for j in pattern for t in decision_periods)
    prob.addConstraint(phi[j,t] <= r[t] for j in pattern for t in decision_periods)
    
    # solve problem
    prob.solve()
    
    # Store the solution in decision space
    x_out = prob.getSolution(x).round()
    y_out = prob.getSolution(y).round()
    phi_out = prob.getSolution(phi).round()
    I_out = prob.getSolution(I)
    r_out = prob.getSolution(r)
    
    # Store the solution in objective space
    f_sol = xp.evaluate(obj,problem=prob)
    
    return x_out,y_out,phi_out,I_out,r_out,f_sol

### Compare the performance of CSRPS and MCSRPS

In [ ]:
# Compare the performance of CSRPS and MCSRPS
group_num = 2
csrps_cal = np.zeros(20)
mcsrps_cal = np.zeros(20)
for jjj in range(1,21):
    example_num = jjj

    # Reading all required components from the data
    with open(f'data/group{group_num}/example{example_num}.txt', mode="r", encoding="utf-8") as fp:
        info_data = fp.read()

    number_of_decision_periods = int(re.findall('periods.*?(\d+)\n', info_data)[0])

    number_of_item_types = int(re.findall('types.*?(\d+)\n',info_data)[0])

    number_of_affected_periods = int(re.findall('affected\speriods.*?(\d+)\n',info_data)[0])

    affect_periods = re.findall('Affected periods:\t(\d+.*\d+)\t\n',info_data)[0].split('\t')
    affected_periods = np.array([int(i) for i in affect_periods])

    Length_of_the_object  = int(re.findall('Length.*?object:\t(\d+).*',info_data)[0])

    leng_items =re.findall('Length.*items:\t(\d+.*\d+).*',info_data)[0].split('\t')
    Length_of_items = np.array([int(j) for j in leng_items])

    Predict_demand = re.findall('Predicted.*items:\n(\d+.*\d+).*Realistic',info_data,re.S)[0].split('\t')
    Predicted_demand_of_items = np.array([int(k) for k in Predict_demand]).reshape(number_of_decision_periods,number_of_item_types)

    Real_demand = re.findall('Realistic.*items:\n(\d+.*\d+).*Consumed',info_data,re.S)[0].split('\t')
    Realistic_demand_of_items = np.array([int(l) for l in Real_demand]).reshape(number_of_decision_periods,number_of_item_types)

    Consumed_cost_per_object  = int(re.findall('Consumed.*?object:\t(\d+).*', info_data)[0])

    Pattern_setup_cost = int(re.findall('Pattern\ssetup.*?:\t(\d+).*', info_data)[0])

    Pattern_deviation_cost = int(re.findall('Pattern\sdeviation.*?:\t(\d+).*', info_data)[0])

    Period_dev = re.findall('Period\sdeviation.*:\t(\d.*\d).*',info_data)[0].split('\t')
    Period_deviation_cost = np.array([float(m) for m in Period_dev])

    Inventory_hold = re.findall('Inventory.*item:\t(\d.*\d).*',info_data)[0].split('\t')
    Inventory_holding_cost_per_item = np.array([float(o) for o in Inventory_hold])

    num_ava_objects = re.findall('.*available.*period:\t(\d.*\d).*',info_data)[0].split('\t')
    Number_of_available_objects_in_each_period  = np.array([float(u) for u in num_ava_objects])

    # Find all possible patterns
    pattern_combination = pattern_comb(Length_of_items,Length_of_the_object)

    # Construct the a matrix
    number_of_possible_pattern = len(pattern_combination)
    a =np.array([0*Length_of_items])
    for i in pattern_combination:
        number_of_each_pattern = np.array([0*Length_of_items])
        for j in range(number_of_item_types):
            for k in i:
                if Length_of_items[j] == k:
                    number_of_each_pattern[0,j] += 1
        a=np.concatenate((a,number_of_each_pattern),axis=0)
    a = a[1:]


    # Define required variables
    items = range(number_of_item_types)
    decision_periods = range(number_of_decision_periods)
    pattern = range(number_of_possible_pattern)
    h = Inventory_holding_cost_per_item
    epsilon = Period_deviation_cost
    l = Length_of_items

    # Calculate csrps performance (obj)
    total_loss = 0
    for kkk in decision_periods:
        total_loss += csrps(kkk)[3]
    csrps_cal[jjj-1] = total_loss

    # Calculate mcsrps performance (obj)
    mcsrps_cal[jjj-1] = mcsrps()[5]

In [ ]:
indices = np.arange(20)
plt.bar(indices,csrps_cal,label="csrps",width=0.8,align="center")
plt.bar(indices,mcsrps_cal,label="mcsrps",width=0.8/2,align="center")
plt.xlabel("Data number")
plt.ylabel("Total cost")
plt.title("Company7")
plt.legend()
plt.show

In [ ]:
sum(csrps_cal)-sum(mcsrps_cal)

In [ ]:
plt.plot(np.arange(20),csrps_cal,label="csrps")
plt.plot(np.arange(20),mcsrps_cal,label="mcsrps")
plt.xlabel("Data number")
plt.ylabel("Total cost")
plt.title("Company1")
plt.legend()
plt.show